# Validating LLM Model Responses Using Pydantic Models
---

### Overview

The project represents a brief introduction to **Pydantic Data Models** and their use in LLM output validation


## Import libraries

In [3]:
from pydantic import BaseModel, ValidationError, EmailStr, Field
from typing import Optional, List, Literal
from datetime import date
import json

import openai
from openai import OpenAI
from dotenv import load_dotenv

Initialize Client

In [4]:
# Load environment variables for API access
load_dotenv()
# Initialize OpenAI client
client = openai.OpenAI()

## Pydantic Model Definition 



### Basic Model Definition

In [6]:
## Pydantic model for validating user input
class UserInput(BaseModel):
    name: str
    email: EmailStr
    query: str

In [10]:
# Create model instance
user_iput = UserInput(
    name="John User",
    email="john.user@email.com",
    query="I need to reset my password"
)

In [29]:
## see instantiation failure due to required type error
#user_iput = UserInput(
#    name=2026,
#    email="john User has no email address",
#    query="I need to reset my password"
#)

In [9]:
## see instantiation failure due to EmailStr error
#user_iput = UserInput(
#    name="John User",
#    email="john User has no email address",
#    query="I need to reset my password"
#)

### Error Handling for Different Inputs


In [ ]:
# Function handles validation errors related to user inputs
def validate_user_input(input_data):
    try:
        # Attempt to create model instance
        user_input = UserInput(**input_data) # unpacks keys/fields
        print(f"✅ Valid user input created:")
        print(f"{user_input.model_dump_json(indent=2)}")
        return user_input
    except ValidationError as e:
        # Display validation errors
        print(f"❌ Validation error occurred:")
        for error in e.errors():
            print(f"  - {error['loc'][0]}: {error['msg']}")
        return None
        

**Instantiation Examples**

In [15]:
# Create an instance of UserInput using validate_user_input() function
input_data = {
    "name": "John User", 
    "email": "john.user@email.com",
    "query": "I need to reset my password"
}

user_input = validate_user_input(input_data)

✅ Valid user input created:
{
  "name": "John User",
  "email": "john.user@email.com",
  "query": "I need to reset my password"
}


In [16]:
# instantiation fails due to missing field
input_data = {
    "name": "John User", 
    "email": "john.user@email.com"
}

user_input = validate_user_input(input_data)

❌ Validation error occurred:
  - query: Field required


##|# Model with Optional Fields

In [ ]:
# New UserInput model with optional fields
class UserInput(BaseModel):
    name: str
    email: EmailStr
    query: str
    order_id: Optional[int] = Field(
        None,
        description="5-digit order number (cannot start with 0)",
        ge=10000,
        le=99999
    )
    purchase_date: Optional[date] = None

**Instantiation Examples**

In [18]:
# Instantiation with required fields only
input_data = {
    "name": "John User", 
    "email": "john.user@email.com",
    "query": "I need to reset my password"
}

# Validate input data
user_input = validate_user_input(input_data)
print(user_input)

✅ Valid user input created:
{
  "name": "John User",
  "email": "john.user@email.com",
  "query": "I need to reset my password"
}
name='John User' email='john.user@email.com' query='I need to reset my password'


In [25]:
# Instantiation with required and optional fields
input_data = {
    "name": "John User", 
    "email": "john.user@email.com",
    "query": f"""I need to reset my password associated with my purhcase order""",
    "order_id": 12345,                # "12345" is coerced to numeric type
    "purchase_date": date(2026, 2, 6) #"2026-02-06" is coerced to datetime type
}

# Validate input data
user_input = validate_user_input(input_data)
print(user_input)

✅ Valid user input created:
{
  "name": "John User",
  "email": "john.user@email.com",
  "query": "I need to reset my password associated with my purhcase order"
}
name='John User' email='john.user@email.com' query='I need to reset my password associated with my purhcase order'


### JSON Model Definitions

In [35]:
# Defining user input as JSON object
input_json = '''
{
    "name": "John User", 
    "email": "john.user@email.com",
    "query": "I need to reset my password associated with my purhcase order", 
    "order_id": 12345,                
    "purchase_date": "2026-2-6" 
}
'''

# Parse JSON to dictionary
input_data = json.loads(input_json)
print("Parsed JSON: ", input_data)

# Validate the input data
user_input = validate_user_input(input_data)

Parsed JSON:  {'name': 'John User', 'email': 'john.user@email.com', 'query': 'I need to reset my password associated with my purhcase order', 'order_id': 12345, 'purchase_date': '2026-2-6'}
✅ Valid user input created:
{
  "name": "John User",
  "email": "john.user@email.com",
  "query": "I need to reset my password associated with my purhcase order"
}


### Validate User Input with `UserInput`'s `model_validate_json` Method

In [38]:
# Parse JSON and validate user input data in single step
user_input = UserInput.model_validate_json(input_json)
print(user_input.model_dump_json(indent=2))

{
  "name": "John User",
  "email": "john.user@email.com",
  "query": "I need to reset my password associated with my purhcase order"
}
